In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy
import os.path
import pandas
from simple_dispatch import generatorData
from simple_dispatch import bidStack
from simple_dispatch import dispatch
from simple_dispatch import generatorDataShort

# Additions

New: https://www.wecc.org/Reliability/2022%20Western%20Assessment%20of%20Resource%20Adequacy.pdf (page 14 link)

Old: https://www.wecc.org/Administrative/06_Brown_Retirements_October%202020.pdf

In [2]:
df = pd.read_excel('../Data/2022 WARA Generation Resource List.xlsx', sheet_name='Resource List')
df = df.loc[(df['Country']=='US')&(df['Status']!='Existing')&(df['Generation Type'].isin(['Natural Gas and Other Gases', 'Nuclear', 'Other', 'Biomass', 'Geothermal']))].reset_index(drop=True)
df['Year'] = pd.to_datetime(df['Initial Operating Month-Year']).dt.year
df.groupby(['Year', 'Generation Type'])['Nameplate Capacity'].sum()

Year  Generation Type            
2019  Geothermal                       52.000
2022  Biomass                          51.244
      Geothermal                       29.000
      Natural Gas and Other Gases     157.600
2023  Geothermal                      439.500
      Natural Gas and Other Gases    1022.400
2024  Natural Gas and Other Gases    1209.030
2025  Natural Gas and Other Gases    1334.000
2026  Geothermal                       44.000
      Natural Gas and Other Gases     615.000
2028  Nuclear                         500.000
2029  Natural Gas and Other Gases     635.800
      Nuclear                         720.000
2031  Geothermal                      801.000
2032  Natural Gas and Other Gases      88.000
2033  Other                           402.000
Name: Nameplate Capacity, dtype: float64

In [3]:
egrid_data_xlsx = '../Data/GridInputData/eGRID2019_data.xlsx'

egrid_unt = pandas.read_excel(egrid_data_xlsx, 'UNT'+'19', skiprows=[0])
egrid_unt['orispl_unit'] = egrid_unt.ORISPL.map(str) + '_' + egrid_unt.UNITID.map(str)

egrid_gen = pandas.read_excel(egrid_data_xlsx, 'GEN'+'19', skiprows=[0])
egrid_gen['orispl_unit'] = egrid_gen['ORISPL'].map(str) + '_' + egrid_gen['GENID'].map(str)

egrid_plnt = pandas.read_excel(egrid_data_xlsx, 'PLNT'+'19', skiprows=[0])


In [4]:
egrid_data_xlsx_new = '../Data/GridInputData/eGRID2022_data.xlsx'

egrid_unt_new = pandas.read_excel(egrid_data_xlsx_new, 'UNT'+'22', skiprows=[0])
egrid_unt_new['orispl_unit'] = egrid_unt_new.ORISPL.map(str) + '_' + egrid_unt_new.UNITID.map(str)

egrid_gen_new = pandas.read_excel(egrid_data_xlsx_new, 'GEN'+'22', skiprows=[0])
egrid_gen_new['orispl_unit'] = egrid_gen_new['ORISPL'].map(str) + '_' + egrid_gen_new['GENID'].map(str)

egrid_plnt_new = pandas.read_excel(egrid_data_xlsx_new, 'PLNT'+'22', skiprows=[0])

In [5]:
gd_short = pickle.load(open('../Data/generator_data_short_WECC_2022_local.obj', 'rb'))

In [6]:
unit_drops = pd.read_csv('../Data/scheduled_retirements_2024final.csv', index_col=0)
leaving_gens = egrid_gen[egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit'].values)].loc[:, ['PSTATABB', 'PNAME', 'GENID', 'FUELG1', 'NAMEPCAP', 'GENYRRET', 'orispl_unit']].copy(deep=True)
staying_gens = egrid_gen.loc[(egrid_gen['orispl_unit'].isin(gd_short.df['orispl_unit'].values))&(~(egrid_gen['orispl_unit'].isin(leaving_gens['orispl_unit'].values)))].loc[:, ['PSTATABB', 'PNAME', 'GENID', 'FUELG1', 'NAMEPCAP', 'PRMVR', 'GENYRONL', 'GENYRRET', 'orispl_unit', 'ORISPL']].copy(deep=True)
staying_gens.head()

,PSTATABB,PNAME,GENID,FUELG1,NAMEPCAP,PRMVR,GENYRONL,GENYRRET,orispl_unit,ORISPL
1331,AZ,Arlington Valley Energy Facility,CTG1,NG,198.0,CT,2002.0,NaN,55282_CTG1,55282
1332,AZ,Arlington Valley Energy Facility,CTG2,NG,198.0,CT,2002.0,NaN,55282_CTG2,55282
1358,AZ,Black Mountain Generating Station,1,NG,60.5,GT,2008.0,NaN,56482_1,56482
1359,AZ,Black Mountain Generating Station,2,NG,60.5,GT,2008.0,NaN,56482_2,56482
1481,AZ,H Wilson Sundt Generating Station,4,NG,173.3,ST,1967.0,NaN,126_4,126


In [7]:
staying_gens['FUELG1'].unique()

array(['NG', 'SUB', 'BIT', 'RC'], dtype=object)

In [8]:
staying_gens['PRMVR'].unique()

array(['CT', 'GT', 'ST', 'CA', 'CS'], dtype=object)

In [9]:
for i in staying_gens.index:
    ind = gd_short.df[gd_short.df['orispl_unit']==staying_gens.loc[i, 'orispl_unit']].index
    staying_gens.loc[i, 'gd_mw'] = gd_short.df.loc[ind, 'mw'].values[0]

### 2019 to 2022

In [10]:
gd_short_2019 = pickle.load(open('../Data/generator_data_short_WECC_2019_local.obj', 'rb'))

In [11]:
new_units_gd2022 = gd_short.df[~gd_short.df.orispl_unit.isin(gd_short_2019.df.orispl_unit.values)]['orispl_unit'].values

In [12]:
new_units_gd2022

array(['10776_GTG', '141_7', '141_8', '2444_6', '315_CT1', '315_CT2',
       '335_CT1', '335_CT2', '377_4', '55129_DBG4', '55129_DBG5',
       '55207_CT7', '55207_CT8', '55372_CTG1', '55372_CTG2', '55372_CTG3',
       '55479_1', '56298_CT005', '56298_CT006', '56319_1', '56596_1',
       '60698_D1', '60698_D7', '7504_1'], dtype=object)

In [14]:
egrid_unt_new.loc[(egrid_unt_new['orispl_unit'].isin(new_units_gd2022))&(egrid_unt_new['UNTYRONL']>=2019)]

,SEQUNT22,YEAR,PSTATABB,PNAME,ORISPL,UNITID,PRMVR,UNTOPST,CAMDFLAG,PRGCODE,...,NOXANSRC,NOXOZSRC,SO2SRC,CO2SRC,HGSRC,SO2CTLDV,NOXCTLDV,HGCTLDV,UNTYRONL,orispl_unit
1004,1005,2022,AZ,Agua Fria Generating Station,141,7,GT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"H2O,SCR",NaN,2022.0,141_7
1005,1006,2022,AZ,Agua Fria Generating Station,141,8,GT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"H2O,SCR",NaN,2022.0,141_8
1090,1091,2022,AZ,Desert Basin Generating Station,55129,DBG4,GT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"H2O,SCR",NaN,2022.0,55129_DBG4
1091,1092,2022,AZ,Desert Basin Generating Station,55129,DBG5,GT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"H2O,SCR",NaN,2022.0,55129_DBG5
1340,1341,2022,CA,AES Alamitos,315,CT1,CT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"DLNB,SCR",NaN,2019.0,315_CT1
1341,1342,2022,CA,AES Alamitos,315,CT2,CT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"DLNB,SCR",NaN,2019.0,315_CT2
1348,1349,2022,CA,AES Huntington Beach,335,CT1,CT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"DLNB,SCR",NaN,2019.0,335_CT1
1349,1350,2022,CA,AES Huntington Beach,335,CT2,CT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"DLNB,SCR",NaN,2019.0,335_CT2
3404,3405,2022,CA,Roseville Energy Park,56298,CT005,GT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"H2O,SCR",NaN,2022.0,56298_CT005
3405,3406,2022,CA,Roseville Energy Park,56298,CT006,GT,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,"H2O,SCR",NaN,2022.0,56298_CT006


- Added Agua Fria 7 and 8 in 2022: NAMEPCAP 99, GT NG (141_7 and 141_8 in unt, 141_AF7 and 141_AF8 in gen)
- Added Desert Basin 4 and 5 in 2022: NAMEPCAP 99, GT NG (55129_DBG4 and 55129_DBG5 in unt, 55129_CTG4 and 55129_CTG5 in gen)
- Added AES Alamitos 315_CT1 and 315_CT2 in 2019, but they're not in gen -> use gd.mw?
- Added AES Huntington 335_CT1 and 335_CT2 in 2019 but they're also not in gen
- Added Roseville Energy Park 56298_CT005 and 56298_CT006 (unt), 56298_0005 in gen, in 2022 (GT): NAMEPCAP 30 GT NG
- Added Stanton 60698 D1 and D7 GTs in 2020: NAMEPCAP 141 GT NG

In [15]:
gd_short.df[gd_short.df['orispl_unit'].isin(['315_CT1', '315_CT2', '335_CT1', '335_CT2'])]['mw'].sum()

1403.0

Total: add 141 GT NG in 2020 and add 1403 CT NG in 2019; assume 2022 is covered in the 2022 additions from the document, included below.

In [16]:
total_added = 0
additions_df = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
test = pd.DataFrame({'Year':5*[2019], 'Fuel Type':5*['NG'], 'PRMVR':5*['CT'], 'orispl_unit':['469_5', '404_4', '2336_8', '330_7', '57978_CT1']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
test = pd.DataFrame({'Year':2*[2019], 'Fuel Type':2*['NG'], 'PRMVR':2*['GT'], 'orispl_unit':['55538_CTG1', '58562_GT-3']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
print(total_added)

1540.0


### 2022 onward

In [17]:
# gt for 2022
test = pd.DataFrame({'Year':2*[2022], 'Fuel Type':2*['NG'], 'PRMVR':2*['GT'], 'orispl_unit':['116_GT6', '55522_CT10']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# st for 2022 - standin for biomass since no staying gens to use
test = pd.DataFrame({'Year':[2022], 'Fuel Type':['NG'], 'PRMVR':['ST'], 'orispl_unit':['2450_1']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# gt for 2023
test = pd.DataFrame({'Year':2*[2023], 'Fuel Type':2*['NG'], 'PRMVR':2*['GT'], 'orispl_unit':['116_GT4', '116_GT7']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# cc and ct together for 2023
test = pd.DataFrame({'Year':2*[2023], 'Fuel Type':2*['NG'], 'PRMVR':2*['CT'], 'orispl_unit':['55985_CTG1', '55985_CTG2']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# gt for 2023
test = pd.DataFrame({'Year':2*[2023], 'Fuel Type':2*['NG'], 'PRMVR':2*['GT'], 'orispl_unit':['6112_5', '55505_BR2']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# 570 + 219 + 233
# gt for 2024
test = pd.DataFrame({'Year':[2024], 'Fuel Type':['NG'], 'PRMVR':['GT'], 'orispl_unit':['56482_1']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# ct for 2024
test = pd.DataFrame({'Year':5*[2024], 'Fuel Type':5*['NG'], 'PRMVR':5*['CT'], 
                     'orispl_unit':['55700_CTG1', '7307_5', '55283_2', '330_5', '330_7']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# gt for 2025
test = pd.DataFrame({'Year':7*[2025], 'Fuel Type':7*['NG'], 'PRMVR':7*['GT'], 'orispl_unit':['55802_CTG1', '2444_9', '56482_1', '56482_2', '55522_CT10', '55698_HEP1', '55698_HEP2']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# cc for 2025
test = pd.DataFrame({'Year':3*[2025], 'Fuel Type':3*['NG'], 'PRMVR':3*['CT'], 'orispl_unit':['55970_2', '330_7', '330_5']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# cc for 2026
test = pd.DataFrame({'Year':3*[2026], 'Fuel Type':3*['NG'], 'PRMVR':3*['CT'], 'orispl_unit':['55748_CTG2', '56041_M1', '56041_M2']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# gt for 2026
test = pd.DataFrame({'Year':5*[2026], 'Fuel Type':5*['NG'], 'PRMVR':5*['GT'], 'orispl_unit':['116_GT3', '116_GT4', '116_GT6', '7975_1', '7975_2']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# cc, ct for 2029
test = pd.DataFrame({'Year':2*[2029], 'Fuel Type':2*['NG'], 'PRMVR':2*['CT'], 'orispl_unit':['57978_CT1', '404_4']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# st for 2029
test = pd.DataFrame({'Year':2*[2029], 'Fuel Type':2*['SUB'], 'PRMVR':2*['ST'], 'orispl_unit':['2450_2', '389_4']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
# gt for 2032
test = pd.DataFrame({'Year':2*[2032], 'Fuel Type':2*['NG'], 'PRMVR':2*['GT'], 'orispl_unit':['56482_1', '7975_4']})
total_added += staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

total_added

6686.0

In [18]:
additions_df[~additions_df['orispl_unit'].isin(gd_short.df['orispl_unit'].values)] # did we accidentally use with mismatch unit names?

,Year,Fuel Type,PRMVR,Large Region,orispl_unit


In [19]:
additions_df[~additions_df['orispl_unit'].isin(staying_gens['orispl_unit'].values)] # did we accidentally use any that will be retired? 

,Year,Fuel Type,PRMVR,Large Region,orispl_unit


In [20]:
additions_df.to_csv('../Data/generator_additions_2024final.csv')

### Other additions:
#### New nuclear and non-emitting gas technologies are unknown
#### New geothermal implemented like new hydro in future_grid.py

New nuclear 500.000 in 2028 and 720 in 2029


New zero emissions gas 402 in 2033


New geothermal 29 in 2022, 439.5 in 2023, 44 in 2026, 801 in 2031

In [44]:
# geothermal namepcap in main wecc states
egrid_gen.loc[(egrid_gen['FUELG1'].isin(['GEO']))&(egrid_gen['PSTATABB'].isin(['CA', 'OR', 'WA', 'NV', 'ID', 'UT', 'AZ', 'WY', 'MT', 'CO', 'NM']))]['NAMEPCAP'].sum()

3838.3999999999996

In [45]:
# biomass namepcap in main wecc states
egrid_gen.loc[(egrid_gen['FUELG1'].isin(['AB', 'LFG', 'MSW', 'OBG' ,'OBL', 'OBS', 'OG', 'OTH', 'WDL', 'WDS']))&(egrid_gen['PSTATABB'].isin(['CA', 'OR', 'WA', 'NV', 'ID', 'UT', 'AZ', 'WY', 'MT', 'CO', 'NM']))]['NAMEPCAP'].sum()

1924.6

In [46]:
3838.4 / (3838.4 + 1924.6) # approximately 2/3 of GEO + BIOMASS total namepcap is GEO

0.6660419920180461

### Small coal special cases - no existing coal small enough to just duplicate

'Deseret Generation & Transmission (Bonanza) (DGT EEI)' built in 2020 retired in 2024
'Tata' coal built 2020 retired in 2023

-> implemented in future_grid.py